In [ ]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [10]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_full_pastis.ipynb


In [11]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [12]:
from scipy import spatial
from sklearn.decomposition import PCA

In [13]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [14]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [15]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [16]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [17]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,1389.13,0.00,0.02,0.00,0.00,-0.0,0.00
3,0.00,440.58,-0.00,-0.01,-0.00,0.0,0.00
4,0.02,-0.00,109.20,-0.00,-0.00,-0.0,-0.00
5,0.00,-0.01,-0.00,20.32,-0.00,0.0,-0.00
6,0.00,-0.00,-0.00,-0.00,3.32,0.0,0.00
7,-0.00,0.00,-0.00,0.00,0.00,0.5,0.00
8,0.00,0.00,-0.00,-0.00,0.00,0.0,0.05


In [18]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,0.00000,0.00006,0.00001,0.00005,-0.00004,0.00005
3,0.00000,1.00000,-0.00002,-0.00005,-0.00004,0.00001,0.00001
4,0.00006,-0.00002,1.00000,-0.00003,-0.00003,-0.00004,-0.00001
5,0.00001,-0.00005,-0.00003,1.00000,-0.00002,0.00001,-0.00001
6,0.00005,-0.00004,-0.00003,-0.00002,1.00000,0.00002,0.00001
7,-0.00004,0.00001,-0.00004,0.00001,0.00002,1.00000,0.00005
8,0.00005,0.00001,-0.00001,-0.00001,0.00001,0.00005,1.00000


In [19]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.037090234233474136

In [20]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.76567233e+05 2.69516338e+04 1.58909321e+03 5.16559269e+01
 1.23032112e+00 2.09206019e-02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998208,-0.058392,-0.012852,-0.002376,-0.000386,-0.000059,-0.000005
1,0.057602,0.996863,-0.053454,-0.009247,-0.001509,-0.000227,-0.000021
2,0.015770,0.052104,0.997302,-0.048639,-0.007621,-0.001148,-0.000105
3,0.003637,0.011502,0.047536,0.997181,-0.056201,-0.008166,-0.000747
4,0.000787,0.002493,0.010055,0.054957,0.995443,-0.076940,-0.006801
5,0.000179,0.000563,0.002272,0.012197,0.075413,0.992835,-0.091857
6,0.000033,0.000103,0.000418,0.002243,0.013713,0.091056,0.995749


In [21]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.0017919 , 0.00313655, 0.00269796, 0.00281947, 0.00455749,
       0.00716519, 0.00425129])

In [22]:
X = np.zeros((bootstrap_size, len(GROUPS)))
X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)